Loading data

In [67]:
%%bash

wget -nv https://github.com/sebastian-lapuschkin/lrp_toolbox/blob/master/data/MNIST/train_images.npy?raw=true -O data/train_images.npy

wget -nv https://github.com/sebastian-lapuschkin/lrp_toolbox/blob/master/data/MNIST/train_labels.npy?raw=true -O data/train_labels.npy

2022-04-30 20:06:38 URL:https://raw.githubusercontent.com/sebastian-lapuschkin/lrp_toolbox/master/data/MNIST/train_images.npy [47040080/47040080] -> "data/train_images.npy" [1]
2022-04-30 20:06:39 URL:https://raw.githubusercontent.com/sebastian-lapuschkin/lrp_toolbox/master/data/MNIST/train_labels.npy [480080/480080] -> "data/train_labels.npy" [1]


In [1]:
import numpy as np
import mlp_utils
from mlp_utils import Dense, ReLU, LogSoftMax, NNetwork
from tqdm import tqdm
from importlib import reload

reload(mlp_utils)

<module 'mlp_utils' from '/home/whoisjiji/Desktop/crafted_neural_networks/mlp_utils.py'>

In [2]:
X = np.load('data/train_images.npy').astype(float) / 255.
Y = np.load('data/train_labels.npy').astype(np.int32)
nclasses = len(np.unique(Y))

In [3]:
def iterate_minibatches(inputs, labels, batchsize, shuffle=True):
    if shuffle:
        indices = np.random.permutation(len(labels))
    else:
        indices = np.arange(len(labels))
    for start_idx in range(0, len(labels), batchsize):
        batch_indices = indices[start_idx : start_idx + batchsize]
        
        yield inputs[batch_indices], labels[batch_indices]

In [6]:
LR = 0.1
input_size = np.prod(X.shape[1:]) ## skipping batch dim
model = NNetwork(layers = [
    Dense(input_size, 2**7, LR),
    ReLU(),
    Dense(2**7, 2**5, LR),
    ReLU(),
    Dense(2**5,nclasses, LR),
],
    loss = LogSoftMax()
)
BATCHSIZE = 32
rand_batch_indices = np.random.choice(len(X), BATCHSIZE)
(model.predict(X[rand_batch_indices]) == Y[rand_batch_indices]).mean()

0.06640625

In [7]:
from IPython.display import clear_output

EPOCHS = 3

for epoch in tqdm(range(EPOCHS)):
    
    for x_batch, y_batch in iterate_minibatches(X, Y, BATCHSIZE):
        layer_inputs, logits = model.forward(x_batch) # FORWARD PASS
        
        # ENDING FORWARD (COMPUTING LOSS) + STARTING BACKWARD (COMPUTING LOGITS GRAD)
        batch_loss, grad_logits = model.compute_loss(logits, y_batch)
        
        model.backward(layer_inputs, grad_logits)  # BACKWARD (UPDATING weights)

  0%|                                                                                             | 0/3 [00:00<?, ?it/s]/home/whoisjiji/Desktop/crafted_neural_networks/mlp_utils.py:62: RuntimeWarning: overflow encountered in exp
  log_sum_logits = np.log(np.sum(np.exp(logits), axis=-1))  # summation over logits axis
/home/whoisjiji/Desktop/crafted_neural_networks/mlp_utils.py:69: RuntimeWarning: overflow encountered in exp
  softmax = np.exp(logits) / np.sum(np.exp(logits),axis=-1, keepdims=True) # summation over logits axis
/home/whoisjiji/Desktop/crafted_neural_networks/mlp_utils.py:69: RuntimeWarning: invalid value encountered in true_divide
  softmax = np.exp(logits) / np.sum(np.exp(logits),axis=-1, keepdims=True) # summation over logits axis
 33%|████████████████████████████▎                                                        | 1/3 [00:06<00:12,  6.39s/it]


KeyboardInterrupt: 